# 虚拟电厂单用户调度（含储能）
- 光储荷经济性调度，针对单个用户、单个光伏、单个储能设备，短周期内的经济最优目标

# 变量
- 充电变量：$X = (x_t) \in [0, \alpha]$
- 放电变量：$Y = (y_t) \in [-\alpha, 0]$
- 储能设备能量状态：$S = (s_t) \in [0, 1]$

以上三个变量为 `continuous_var_list` 连续变量列表

# 参数
- 负荷：`load`
- 功率：`power`
- 电价：`price`
- 充电效率：`charge_efficiency`
- 放电效率：`discharge_efficiency`
- 储能单次充放电限制：`nominal_power`

# 目标函数
最小化经济性最优目标：
$$
\text{minimize } Cost = \sum_{t=1}^T (load(t) - power(t) + x(t) + y(t)) \times price(t) \div costbase
$$

其中 `costbase` 为不使用储能的情况下的花费：
$$
costbase = \sum_{t=1}^T (load(t) - power(t)) \times price(t)
$$

# 约束条件
- 储能能量状态：
  - 储能初始状态为 0：$s_0 = 0$
  - 储能状态更新：$s_t = s_{t-1} + c_1 \cdot x_t + \frac{y_t}{c_2}$ 公式错误


In [1]:
import cplex
import docplex
import os
print(os.path.dirname(cplex.__file__))

e:\Pythonworkshop\optimization_test\.conda\lib\site-packages\cplex


In [2]:
from docplex.mp.model import Model
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np




# 设置一个随机种子
random.seed(1234)


clear_interval = 30  # 出清步长，单位分钟
clear_period = 24 * 600  # 优化周期一天，共有 1440 分钟
ctrl_interval = 1  # 控制变量步长，单位s

# 验证 clear_period 是否是 clear_interval 的倍数
if clear_period % clear_interval != 0 or clear_interval % ctrl_interval !=0:
    raise ValueError("clear_period 必须是 clear_interval 的整数倍")

n_clearance = clear_period // clear_interval  # 总出清次数
n_ctrl = clear_period // ctrl_interval  # 控制变量次数

# 生成随机的用户负荷和用户功率
user_loads = [round(random.uniform(0, 1), 2) for _ in range(n_clearance)]
user_powers = [round(random.uniform(0, 1), 2) for _ in range(n_clearance)]
user_loads = np.repeat(user_loads,clear_interval/ctrl_interval)
user_powers = np.repeat(user_powers,clear_interval/ctrl_interval)
netload = [user_loads[i] - user_powers[i] for i in range(n_ctrl)]
print("净负荷：", netload)

# 生成随机电价，每4小时变动一次，基于 0.5 生成 ±0.2 的波动
elec_price = []
for i in range(0, n_clearance, 12):  # 每4小时（12个采样点）生成一个电价
    price = round(0.5 + random.uniform(-0.2, 0.2), 2)
    elec_price.extend([price] * 12)
elec_price = np.repeat(elec_price,clear_interval/ctrl_interval)
print('电价：', elec_price)

# 储能系统参数
charge_eff = 0.91
discharge_eff = 0.95
nominal_power = 0.8  # BESS 额定最大功率
SOC_ub = 1  # SOC 上限
SOC_lb = 0  # SOC 下限
SOC0 = 0.5  # 初始 SOC
Ckwh = 1    # 电池容量（kWh）

# 创建模型
model = Model(name='Electricity Optimization')
model.parameters.threads.set(0)

# 创建变量
x = model.continuous_var_list(n_ctrl, lb=0, ub=nominal_power, name='x')  # 充电功率
y = model.continuous_var_list(n_ctrl, lb=-nominal_power, ub=0, name='y')  # 放电功率
soc = model.continuous_var_list(n_ctrl, lb=SOC_lb, ub=SOC_ub, name='soc')  # 电池SOC
z = model.binary_var_list(n_ctrl, name='z')  # 二进制变量，表示充电或放电状态

# 无储能下的花费
cost_base = sum(((user_loads[i] - user_powers[i]) * elec_price[i]) for i in range(n_ctrl))
print("无储能下的花费：", cost_base)

# 目标：最小化总花费
total_cost = model.sum(((user_loads[i] - user_powers[i] + x[i] + y[i]) * elec_price[i]) for i in range(n_ctrl))
model.minimize(total_cost)

# 初始 SOC 约束
model.add_constraint(soc[0] == SOC0)

# 设置 SOC 更新约束
for i in range(n_ctrl - 1):
    # SOC 状态更新约束
    #model.add_constraint(soc[i + 1] == soc[i] + x[i] * charge_eff / 60 / Ckwh * ctrl_interval + y[i] / discharge_eff / 60 / Ckwh * ctrl_interval)
    model.add_constraint(x[i + 1] <=  x[i] +0.01)
    model.add_constraint(x[i + 1] >=  x[i] -0.01)
    model.add_constraint(y[i + 1] <=  y[i] +0.01)
    model.add_constraint(y[i + 1] >=  y[i] -0.01)
   


    model.add_constraint(soc[i + 1] == soc[i] + (x[i] * charge_eff / 60 / Ckwh * ctrl_interval) + (y[i] / discharge_eff / 60 / Ckwh * ctrl_interval))
    #model.add_constraint(soc[i + 1] == soc[i] +  (x[i] * charge_efficiency(soc[i]) / 60 / Ckwh * ctrl_interval) + (y[i] / discharge_efficiency(soc[i]) / 60 / Ckwh * ctrl_interval))

for i in range(n_ctrl):
    # 限制充放电不能同时进行
    model.add_constraint(x[i] <= z[i] * nominal_power)  # 当 z[i] = 1 时允许充电
    model.add_constraint(-y[i] <= (1 - z[i]) * nominal_power)  # 当 z[i] = 0 时允许放电

# 解决问题
# 打印模型的变量数和约束数
print("模型的变量数量:", model.number_of_variables)
print("模型的约束数量:", model.number_of_constraints)
solution = model.solve()


# 打印优化后的结果并保存到 Excel
if solution:
    print("优化后的总花费：", solution.objective_value)
    
    # 提取每个时段的充放电决策和SOC
    charge_values = [solution.get_value(x[i]) for i in range(n_ctrl)]
    discharge_values = [solution.get_value(y[i]) for i in range(n_ctrl)]
    soc_values = [solution.get_value(soc[i]) for i in range(n_ctrl)]
    
    # 将数据保存到 pandas DataFrame 中
    results = pd.DataFrame({
        '时间段': range(n_ctrl),
        '用户负荷': user_loads,
        '用户功率': user_powers,
        '电价': elec_price,
        '充电功率': charge_values,
        '放电功率': discharge_values,
        'SOC': soc_values
    })
    
    # 将结果保存为 Excel 文件
    results.to_excel('optimization_results.xlsx', index=False)
    print("优化结果已保存为 'optimization_results.xlsx'")
else:
    print("未找到可行解")


净负荷： [0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.61, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, -0.67, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.15999999999999998, 0.15999999999999998, 0.15999999999999998, 0.15999999999999998, 0.15999999999999998, 0.15999999999999998

DOcplexLimitsExceeded: **** Promotional version. Problem size limits (1000 vars, 1000 consts) exceeded, model has 57600 vars, 100796 consts, CPLEX code=1016

In [4]:
import random
import pandas as pd
import numpy as np
import pulp
from pulp import SCIP_CMD
import tempfile

# 设置随机种子
random.seed(1234)

# 参数设置
clear_interval = 30
clear_period = 24 * 60
ctrl_interval = 1

if clear_period % clear_interval != 0 or clear_interval % ctrl_interval != 0:
    raise ValueError("clear_period 必须是 clear_interval 的整数倍")

n_clearance = clear_period // clear_interval
n_ctrl = clear_period // ctrl_interval

# 生成随机数据
user_loads = [round(random.uniform(0, 1), 2) for _ in range(n_clearance)]
user_powers = [round(random.uniform(0, 1), 2) for _ in range(n_clearance)]
user_loads = np.repeat(user_loads, clear_interval // ctrl_interval)
user_powers = np.repeat(user_powers, clear_interval // ctrl_interval)
netload = [user_loads[i] - user_powers[i] for i in range(n_ctrl)]
print("净负荷：", netload)

elec_price = []
for i in range(0, n_clearance, 12):
    price = round(0.5 + random.uniform(-0.2, 0.2), 2)
    elec_price.extend([price] * 12)
elec_price = np.repeat(elec_price, clear_interval // ctrl_interval)
print('电价：', elec_price)

# 储能系统参数
charge_eff = 0.91
discharge_eff = 0.95
nominal_power = 0.8
SOC_ub = 1
SOC_lb = 0
SOC0 = 0.5
Ckwh = 1

# 创建Pulp模型
model = pulp.LpProblem("Electricity_Optimization", pulp.LpMinimize)

# 变量定义
x = [pulp.LpVariable(f"x_{i}", lowBound=0, upBound=nominal_power) for i in range(n_ctrl)]
y = [pulp.LpVariable(f"y_{i}", lowBound=-nominal_power, upBound=0) for i in range(n_ctrl)]
soc = [pulp.LpVariable(f"soc_{i}", lowBound=SOC_lb, upBound=SOC_ub) for i in range(n_ctrl)]
z = [pulp.LpVariable(f"z_{i}", cat="Binary") for i in range(n_ctrl)]

# 无储能下的花费
cost_base = sum(((user_loads[i] - user_powers[i]) * elec_price[i]) for i in range(n_ctrl))
print("无储能下的花费：", cost_base)

# 目标函数
total_cost = pulp.lpSum(
    (user_loads[i] - user_powers[i] + x[i] + y[i]) * elec_price[i] for i in range(n_ctrl)
)
model += total_cost

# 初始 SOC 约束
model += soc[0] == SOC0

# 添加 SOC 更新和充放电限制约束
for i in range(n_ctrl - 1):
    model += soc[i + 1] == soc[i] + x[i] * (charge_eff * ctrl_interval / 60 / Ckwh) + y[i] * (ctrl_interval / discharge_eff / 60 / Ckwh)
    model += x[i + 1] <= x[i] + 0.01
    model += x[i + 1] >= x[i] - 0.01
    model += y[i + 1] <= y[i] + 0.01
    model += y[i + 1] >= y[i] - 0.01

for i in range(n_ctrl):
    model += x[i] <= z[i] * nominal_power
    model += -y[i] <= (1 - z[i]) * nominal_power

# 创建临时文件来存储日志
with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
    log_file_path = tmp_file.name

# 使用 SCIP 求解器并将输出重定向到日志文件
solution = model.solve(SCIP_CMD(msg=True, logPath=log_file_path))

# 打印 SCIP 求解器日志
with open(log_file_path, 'r') as log_file:
    print(log_file.read())

# 检查解并输出结果
if pulp.LpStatus[model.status] == 'Optimal':
    print("优化后的总花费：", pulp.value(model.objective))
    
    charge_values = [pulp.value(x[i]) for i in range(n_ctrl)]
    discharge_values = [pulp.value(y[i]) for i in range(n_ctrl)]
    soc_values = [pulp.value(soc[i]) for i in range(n_ctrl)]
    
    results = pd.DataFrame({
        '时间段': range(n_ctrl),
        '用户负荷': user_loads,
        '用户功率': user_powers,
        '电价': elec_price,
        '充电功率': charge_values,
        '放电功率': discharge_values,
        'SOC': soc_values
    })
    
    results.to_excel('optimization_results.xlsx', index=False)
    print("优化结果已保存为 'optimization_results.xlsx'")
else:
    print("未找到可行解")


净负荷： [0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.52, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, -0.13, 

UnsupportedOperation: fileno

In [ ]:
# 使用其他求解器示例（如GLPK）
from pulp import GLPK_CMD


# 创建Pulp模型
model_2 = pulp.LpProblem("Electricity_Optimization", pulp.LpMinimize)

# 变量定义
x = [pulp.LpVariable(f"x_{i}", lowBound=0, upBound=nominal_power) for i in range(n_ctrl)]
y = [pulp.LpVariable(f"y_{i}", lowBound=-nominal_power, upBound=0) for i in range(n_ctrl)]
soc = [pulp.LpVariable(f"soc_{i}", lowBound=SOC_lb, upBound=SOC_ub) for i in range(n_ctrl)]
z = [pulp.LpVariable(f"z_{i}", cat="Binary") for i in range(n_ctrl)]

# 无储能下的花费
cost_base = sum(((user_loads[i] - user_powers[i]) * elec_price[i]) for i in range(n_ctrl))
print("无储能下的花费：", cost_base)

# 目标函数
total_cost = pulp.lpSum(
    (user_loads[i] - user_powers[i] + x[i] + y[i]) * elec_price[i] for i in range(n_ctrl)
)
model_2 += total_cost

# 初始 SOC 约束
model_2 += soc[0] == SOC0

# 添加 SOC 更新和充放电限制约束
for i in range(n_ctrl - 1):
    model_2 += soc[i + 1] == soc[i] + x[i] * (charge_eff * ctrl_interval / 60 / Ckwh) + y[i] * (ctrl_interval / discharge_eff / 60 / Ckwh)
    model_2 += x[i + 1] <= x[i] + 0.01
    model_2 += x[i + 1] >= x[i] - 0.01
    model_2 += y[i + 1] <= y[i] + 0.01
    model_2 += y[i + 1] >= y[i] - 0.01

for i in range(n_ctrl):
    model_2 += x[i] <= z[i] * nominal_power
    model_2 += -y[i] <= (1 - z[i]) * nominal_power
solution_glpk = model_2.solve(GLPK_CMD(msg=True))

# 检查解并输出结果
if pulp.LpStatus[model_2.status] == 'Optimal':
    print("优化后的总花费：", pulp.value(model_2.objective))
    
    charge_values = [pulp.value(x[i]) for i in range(n_ctrl)]
    discharge_values = [pulp.value(y[i]) for i in range(n_ctrl)]
    soc_values = [pulp.value(soc[i]) for i in range(n_ctrl)]
    
    results = pd.DataFrame({
        '时间段': range(n_ctrl),
        '用户负荷': user_loads,
        '用户功率': user_powers,
        '电价': elec_price,
        '充电功率': charge_values,
        '放电功率': discharge_values,
        'SOC': soc_values
    })
    
    results.to_excel('optimization_results_GLPK.xlsx', index=False)
    print("优化结果已保存为 'optimization_results_GLPK.xlsx'")
else:
    print("未找到可行解")



无储能下的花费： -31.803


In [ ]:
# 使用 SCIP 求解器
from pulp import SCIP_CMD
solution_SCIP = model.solve(SCIP_CMD(msg=True))

# 检查解并输出结果
if pulp.LpStatus[model.status] == 'Optimal':
    print("优化后的总花费：", pulp.value(model.objective))
    
    charge_values = [pulp.value(x[i]) for i in range(n_ctrl)]
    discharge_values = [pulp.value(y[i]) for i in range(n_ctrl)]
    soc_values = [pulp.value(soc[i]) for i in range(n_ctrl)]
    
    results = pd.DataFrame({
        '时间段': range(n_ctrl),
        '用户负荷': user_loads,
        '用户功率': user_powers,
        '电价': elec_price,
        '充电功率': charge_values,
        '放电功率': discharge_values,
        'SOC': soc_values
    })
    
    results.to_excel('optimization_results_scip.xlsx', index=False)
    print("优化结果已保存为 'optimization_results_scip.xlsx'")
else:
    print("未找到可行解")